LangChain 六大模块

LLM
prompt
chains
retrieval
memory
agents

chain 有两种输入： input (prompt)  或者  dict；

dict 更好用，因为可以有名字

## basic RAG 

In [ ]:
from operator import itemgetter
from dotenv import dotenv_values
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.document_loaders import TextLoader

In [ ]:
ENV_CONFIG = dotenv_values("../.env")
vectorstore = FAISS.load_local("../data/db_index", 
    OpenAIEmbeddings(api_key=ENV_CONFIG.get("API_KEY"), 
                               base_url=ENV_CONFIG.get("BASE_URL"))
)
retriever = vectorstore.as_retriever()

In [ ]:
vectorstore.similarity_search("rag的本质是什么", k=1)

In [ ]:

text_loader = TextLoader("../data/text.txt")
text_docs = text_loader.load()

bm25_retriever = BM25Retriever.from_documents(documents=text_docs, k=1)

In [ ]:

emb_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
bm25_retriever

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, emb_retriever], weights=[0.5, 0.5]
)


In [ ]:
# {"question"： "rag的本质是什么？", "context": "doc1, doc2"}
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:


# template = """Answer the question based only on the following context:
# {context}

# Question: {question}
# """
# prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    api_key=ENV_CONFIG.get("API_KEY"), base_url=ENV_CONFIG.get("BASE_URL")
)

In [ ]:
#  chain也是一个函数， 函数就是一个 DAG
# 解读 Chain； 
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
print(({"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt).invoke("Rag的本质是什么").messages[0].content)

In [ ]:
# type(chain)
model.invoke("rag的本质是什么")


In [ ]:
chain.invoke("rag的本质是什么？")


In [ ]:
type(chain)

In [ ]:
chain.invoke("RAG的本质是什么？")

## 上面是一个简单的 RAG

## langChain 自带的 RAG （工作用，学习不用）

In [ ]:
# ## 用 写好的 chain

# from langchain.chains import RetrievalQA

# qa = RetrievalQA.from_chain_type(llm=model, 
#                                  chain_type="stuff", 
#                                  retriever=retriever

#                                  )
# qa.invoke('rag的本质是什么')

## RAG 复杂化一下

希望传入一点别的东西；
- 希望规定语言是什么
- chain的另外一种写法，（通过 itemgetter 获取 input key value 内容）

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | ensemble_retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "RAG的本质是什么？", "language": "English"})

## format document
先看一下 document 是什么

In [ ]:
docs_test = ensemble_retriever.get_relevant_documents("rag的本质是什么")

In [ ]:
len(docs_test)

In [ ]:
print(docs_test[0].page_content)

In [ ]:
print(docs_test[1].page_content)

## format document


1. 用户可能有多轮
2. 太长了 -> 历史对话总结，
3. 用户说法不规范，-> 格式化/标准化用户文法
4. format document

rag的本质是富尔玛 ->  请问RAG的本质是什么？

In [ ]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

In [ ]:
from langchain.prompts.prompt import PromptTemplate
# 这里 chat history 是用来 【标准化用户的输入的问题】

_template = """Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    # 重新排序； 把第二个放到 最末尾
    final_docs = [docs[0]]
    for i in range(2, len(docs)):
        final_docs.append(docs[i])
    final_docs.append(docs[1])
    
    doc_strings = [format_document(doc, document_prompt) for doc in final_docs]
    return document_separator.join(doc_strings)
# print('\n'.join([x.page_content for x in docs_test]))

In [ ]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0, api_key=ENV_CONFIG.get("API_KEY"), base_url=ENV_CONFIG.get("BASE_URL"))
    | StrOutputParser(),
)

_context = {
    "context": itemgetter("standalone_question") | ensemble_retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI(api_key=ENV_CONFIG.get("API_KEY"), base_url=ENV_CONFIG.get("BASE_URL")) | StrOutputParser()

In [ ]:
# vector store 里面；
# 可以读 mysql 的数据  -> Chroma, Faiss
# conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI(api_key=ENV_CONFIG.get("API_KEY"), base_url=ENV_CONFIG.get("BASE_URL")) | StrOutputParser()


In [ ]:
# # {"context": xxx, "question"： xxxx}
# # 假设要同时返回 retriever 检索回的 document
# conversational_qa_chain2 = _inputs | _context | {
#     "ans": ANSWER_PROMPT | ChatOpenAI(api_key=ENV_CONFIG.get("API_KEY"), base_url=ENV_CONFIG.get("BASE_URL")) | StrOutputParser(),
#     "doc": itemgetter("context")
# }
# conversational_qa_chain2.invoke({"question": "RAG 的本质是什么？",
#         "chat_history": [],})

In [ ]:
# 这里是为了让大家知道 langchain 的每一步都输入什么？（可以自己采用类似的调试方式）
# conversational_qa_chain

# (_inputs).invoke({"question": "RAG 的本质是什么？",
#         "chat_history": [],})

print((_inputs | _context | ANSWER_PROMPT).invoke({"question": "RAG 的本质是什么？",
        "chat_history": [],}))

In [ ]:
(_inputs | _context).invoke({
        "question": "RAG 的本质是什么？",
        "chat_history": [],
    })

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "RAG 的本质是什么？",
        "chat_history": [],
    }
)

## 更复杂的 RAG，把 检索的 document 也返回

In [ ]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [ ]:
# 设置一个 memory; 自动维护，而不是类似于上面的【用户传入 chat_history 、 注意这里的区别】
# 上一个 demo 是用户要输入 history, 这里是通过 memory 维护，而不用每一次 invoke的时候都加上
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# 标准化用户的输入问题
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0, api_key=ENV_CONFIG.get("API_KEY"), base_url=ENV_CONFIG.get("BASE_URL"))
    | StrOutputParser(),
}
# 检索 Document 
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# 通过检索回来的 doc 和标准化后的 question 构建 prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# 返回答案，两条路
# 一条直接 获得检索内容
# 一条 构建 prompt -> chatmodel -> output parser
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(api_key=ENV_CONFIG.get("API_KEY"), base_url=ENV_CONFIG.get("BASE_URL")),
    "docs": itemgetter("docs"),
}
# 构建完整链
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [ ]:
inputs = {"question": "Lewis提出了什么方法？"}
result = final_chain.invoke(inputs)
result

In [ ]:
# 用户的 memory 需要自己存一下；
# 但是相对于手动传 history 好很多了；
memory.save_context(inputs, {"answer": result["answer"].content})

In [ ]:
memory.load_memory_variables({})

In [ ]:
# 这里看到不要穿 history, 因为通过 memory 完成了；
inputs = {"question": "how did Rag really work?"}
result = final_chain.invoke(inputs)
result